In [146]:
import pandas as pd
import numpy as np
import sklearn.datasets 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import _tree
import re
import shap

# Load the Iris dataset
data = sklearn.datasets.load_breast_cancer()
X, y = data.data, data.target

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=5, random_state=42)
model.fit(X, y)

from sklearn import tree


In [156]:
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

shap_importance = abs(shap_values.values).mean(axis=0)  # This extracts feature importances

# Ensure the lengths match
if len(shap_importance) == len(data.feature_names):
    # Create a DataFrame with feature names and their SHAP importances
    importance_df = pd.DataFrame({
        'Feature': data.feature_names,
        'SHAP Importance': shap_importance
    }).sort_values(by='SHAP Importance', ascending=False)
else:
    # Log a message if there's still a mismatch in length
    print(f"Mismatch in lengths: {len(shap_importance)} SHAP values vs. {len(data.feature_names)} feature names.")
    # Inspect the SHAP values or data if needed

# Display the DataFrame
print(importance_df)

ValueError: Per-column arrays must each be 1-dimensional

In [ ]:
feature_importances = pd.DataFrame({
    'Feature': data.feature_names,
    'Importance': np.abs(shap_values.values).mean(axis=0)
})

# Sort features by importance
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

In [14]:
from sklearn.tree import _tree
import numpy as np

def get_rules_from_tree(tree, feature_names, class_names):
    """Extract rules from a single decision tree."""
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " --> "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules

def get_rules_from_forest(forest, feature_names, class_names):
    """Extract rules from each tree in a RandomForestClassifier."""
    all_rules = []
    
    for tree_idx, tree in enumerate(forest.estimators_):
        rules = get_rules_from_tree(tree, feature_names, class_names)
        for rule_idx, rule in enumerate(rules):
            formatted_rule = f"Tree {tree_idx} - Rule {rule_idx}: {rule}"
            all_rules.append(formatted_rule)
        
    return all_rules



In [157]:
rules = get_rules_from_forest(model, data.feature_names, data.target_names)
rules

['Tree 0 - Rule 0: if (worst concave points <= 0.144) and (mean texture <= 21.305) and (worst area <= 947.6) and (perimeter error <= 4.032) and (worst compactness <= 0.349) --> class: benign (proba: 100.0%) | based on 167 samples',
 'Tree 0 - Rule 1: if (worst concave points > 0.144) and (worst area > 492.25) and (perimeter error > 1.54) and (mean compactness > 0.098) --> class: malignant (proba: 100.0%) | based on 109 samples',
 'Tree 0 - Rule 2: if (worst concave points <= 0.144) and (mean texture > 21.305) and (worst perimeter <= 101.3) and (mean smoothness <= 0.089) --> class: benign (proba: 100.0%) | based on 27 samples',
 'Tree 0 - Rule 3: if (worst concave points <= 0.144) and (mean texture > 21.305) and (worst perimeter > 101.3) and (worst radius > 16.8) --> class: malignant (proba: 100.0%) | based on 11 samples',
 'Tree 0 - Rule 4: if (worst concave points <= 0.144) and (mean texture > 21.305) and (worst perimeter <= 101.3) and (mean smoothness > 0.089) and (mean smoothness > 

In [16]:
import sklearn
BC = sklearn.datasets.load_breast_cancer()
df = pd.DataFrame(data=BC.data, columns=BC.feature_names)
df['target'] = BC.target

In [17]:
max_val_dict = df.describe().loc['max']
min_val_dict = df.describe().loc['min']

In [80]:
import re

def extract_conditions_from_rule(rule_list):
    """Extract and combine conditions from the rule list."""
    # Combine the list elements into a single string, except for the last part (class and samples info)
    condition_str = ''.join(rule_list[:-1])
    
    # Remove outer parentheses around each condition
    condition_str = re.sub(r'\s*\(\s*', '(', condition_str)
    condition_str = re.sub(r'\s*\)\s*', ')', condition_str)
    
    # Extract individual conditions from the combined string
    conditions = re.findall(r'\(([^)]+)\)', condition_str)
    
    return conditions

def parse_condition(condition):
    """Parse a condition from a string and return a lambda function."""
    pattern_for_feature = r'\b[A-Za-z_][A-Za-z0-9_ ]*\b'
    pattern_for_symbol = r'[<>=]+'
    pattern_for_value = r'[-+]?\d*\.?\d+'
    
    feature_match = re.search(pattern_for_feature, condition)
    symbol_match = re.search(pattern_for_symbol, condition)
    value_match = re.search(pattern_for_value, condition)
    
    if feature_match and symbol_match and value_match:
        feature = feature_match.group().strip()
        symbol = symbol_match.group()
        value = float(value_match.group())
        #print(feature,symbol,value)
        # Return a function that evaluates the condition
        return lambda data_point: eval(f"data_point.get('{feature}') {symbol} {value}")
    
    return True

def check_conditions(data_point_dict, rule):
    """Check if the data point satisfies all the given conditions."""
    conditions  = extract_conditions_from_rule(rule)
    for condition in conditions:

        condition_func = parse_condition(condition)
        if condition_func is None:
            continue
        if not condition_func(data_point_dict):
            return False
    return True



In [125]:
import re

def extract_conditions_from_rule(rule_list):
    """Extract and combine conditions from the rule list."""
    # Combine the list elements into a single string, except for the last part (class and samples info)
    condition_str = ''.join(rule_list[:-1])
    
    # Remove outer parentheses around each condition
    condition_str = re.sub(r'\s*\(\s*', '(', condition_str)
    condition_str = re.sub(r'\s*\)\s*', ')', condition_str)
    
    # Extract individual conditions from the combined string
    conditions = re.findall(r'\(([^)]+)\)', condition_str)
    
    return conditions

def parse_condition(condition):
    """Parse a condition from a string and return a lambda function."""
    pattern_for_feature = r'\b[A-Za-z_][A-Za-z0-9_ ]*\b'
    pattern_for_symbol = r'[<>=]+'
    pattern_for_value = r'[-+]?\d*\.?\d+'
    
    feature_match = re.search(pattern_for_feature, condition)
    symbol_match = re.search(pattern_for_symbol, condition)
    value_match = re.search(pattern_for_value, condition)
    
    if feature_match and symbol_match and value_match:
        feature = feature_match.group().strip()
        symbol = symbol_match.group()
        value = float(value_match.group())
        
        # Return a function that evaluates the condition
        return lambda data_point: eval(f"data_point.get('{feature}') {symbol} {value}")
    
    return lambda data_point: True

def check_conditions(data_point_dict, rule):
    """Check if the data point satisfies all the given conditions."""
    conditions = extract_conditions_from_rule(rule)
    for condition in conditions:
        condition_func = parse_condition(condition)
        if not condition_func(data_point_dict):
            return False
    return True

In [126]:
def get_neigbors(rule):
    neighbors = []
    for i in range(len(X)):
        if check_conditions(df.iloc[i],rule):
           neighbors.append(i)
    return neighbors

In [131]:
def get_bounds(neighborhood, rule):
    condition_bound_dict= {}
    conditions = extract_conditions_from_rule(rule)[:-1]
    
    for condition in conditions:
        #print(condition)
        pattern_for_feature = r'\b[A-Za-z_][A-Za-z0-9_ ]*\b'
        feature_match = re.search(pattern_for_feature, condition)
        feature = feature_match.group().strip()
        #print(feature)
        condition_bound_dict[f'{feature}'] = {}
        #print(condition_bound_dict)
        max = 0
        min= float('inf')
        for neigbor in neighborhood:
            neigbor= df.iloc[neigbor]
            #print(neigbor[f'{feature}'])
            if neigbor[f'{feature}'] > max:
                max = neigbor[f'{feature}'] 
            if neigbor[f'{feature}'] < min:
                min = neigbor[f'{feature}']
        condition_bound_dict[f'{feature}'] = {'max' : max , 'min' : min}

    return condition_bound_dict

In [132]:
N = get_neigbors(rules[0])
B = get_bounds(N ,rules[0])
print(B)

{'worst concave points': {'max': 0.1427, 'min': 0.0}, 'mean texture': {'max': 21.28, 'min': 9.71}, 'worst area': {'max': 932.7, 'min': 185.2}, 'perimeter error': {'max': 4.021, 'min': 0.757}, 'worst compactness': {'max': 0.3441, 'min': 0.02729}}


In [170]:
def update_rule(rule, thresholds):
    # Extract feature names from the rule
    features = [match.group(0) for match in re.finditer(r'\((\w+ \w+)\s<=', rule)]
    print(features)
    # Update conditions based on thresholds
    updated_conditions = []
    for feature in features:
        feature_key = ' '.join(feature.split()[:2])
        if feature_key in thresholds:
            min_val = thresholds[feature_key]['min']
            max_val = thresholds[feature_key]['max']
            condition = f"({min_val}< {feature_key} <= {max_val})"
            updated_conditions.append(condition)
        
    
    # Replace old conditions with new conditions
    for feature, condition in zip(features, updated_conditions):
        rule = re.sub(rf'\({feature} <= [\d.]+\)', condition, rule)
    
    return rule
    

In [171]:
update_rule(rules[0],B)

['(mean texture <=', '(worst area <=', '(perimeter error <=', '(worst compactness <=']


'Tree 0 - Rule 0: if (worst concave points <= 0.144) and (mean texture <= 21.305) and (worst area <= 947.6) and (perimeter error <= 4.032) and (worst compactness <= 0.349) --> class: benign (proba: 100.0%) | based on 167 samples'